In [11]:
from dotenv import dotenv_values
from openai import OpenAI
from pathlib import Path
import json
from nbconvert import HTMLExporter
from nbconvert.writers import FilesWriter
import nbformat

In [12]:
env = dotenv_values(Path(".") / ".." / ".env")

In [13]:
openai_client = OpenAI(api_key=env["OPENAI_API_KEY"])

In [14]:
PROJECTS_PATH = Path(".") / ".." / "docs" / "projects"

In [15]:
# PAGE_PATH =  PROJECTS_PATH / "titanic" / "history_of_survivability_and_secrets_of_shipbuilding"
# NOTEBOOK_NAME = "titanic.ipynb"
# NOTEBOOK_PATH = PAGE_PATH / NOTEBOOK_NAME
# HTML_NAME = "titanic.html"
# HTML_PATH = PAGE_PATH / HTML_NAME

# with open(PAGE_PATH / "notes.txt", "r") as f:
#     NOTES = f.read()

# AUTHOR_NAME = "Rafal Ney"
# AUTHOR_LINK = "/od-zera-do-ai-portfolio/uczestnicy/rafal_ney"

In [ ]:
# PAGE_PATH =  PROJECTS_PATH / "titanic" / "in_depth_analysis_eda"
# NOTEBOOK_NAME = "titanic.ipynb"
# NOTEBOOK_PATH = PAGE_PATH / NOTEBOOK_NAME
# HTML_NAME = "titanic.html"
# HTML_PATH = PAGE_PATH / HTML_NAME

# with open(PAGE_PATH / "notes.txt", "r") as f:
#     NOTES = f.read()

# AUTHOR_NAME = "Natalia Swojak"
# AUTHOR_LINK = "/od-zera-do-ai-portfolio/uczestnicy/natalia_swojak"

## Konwertuj notebook -> html 

In [17]:
with open(NOTEBOOK_PATH) as f:
    notebook = nbformat.read(f, as_version=4)

html_exporter = HTMLExporter()
(html_body, resources) = html_exporter.from_notebook_node(notebook)

with open(HTML_PATH, 'w') as f:
    f.write(html_body)

## Opis projektu

In [7]:
def add_title_and_description(text):

    prompt = f"""
Wygeneruj tytuł i opis do strony prezentującej projekt.
Opis ma być wstępem przygotowującym czytelnika do zapoznania się z samym projektem.

{text}
    Zwróć obiekt JSON z kluczami "title" i "description".
    """

    response = openai_client.chat.completions.create(
        messages=[
            {"role": "user", "content": prompt},
        ],
        model="gpt-4o",
        temperature=0,
    )

    result = response.choices[0].message.content.replace("```json", "").replace("```", "").strip()

    return json.loads(result)

In [7]:
title_and_desc = add_title_and_description(NOTES)

## Wygeneruj index.md

In [10]:
def render_content(title, description, author_name, author_link, notebook_link, html_link):

    return f"""
# {title}

{description}

**Autor**: [{author_name}]({author_link})

<a href="{notebook_link}" class="md-button md-button--primary">Pobierz Notebook</a>

<iframe
    id="content"
    src="{html_link}"
    width="100%"
    style="border:1px solid black;overflow:hidden;"
></iframe>
<script>
function resizeIframeToFitContent(iframe) {{
    iframe.style.height = (iframe.contentWindow.document.documentElement.scrollHeight + 50) + "px";
    iframe.contentDocument.body.style["overflow"] = 'hidden';
}}
window.addEventListener('load', function() {{
    var iframe = document.getElementById('content');
    resizeIframeToFitContent(iframe);
}});
window.addEventListener('resize', function() {{
    var iframe = document.getElementById('content');
    resizeIframeToFitContent(iframe);
}});
</script>
"""

In [11]:
with open("index.md", "w") as f:
    f.write(render_content(
        title=title_and_desc["title"],
        description=title_and_desc["description"],
        author_name=AUTHOR_NAME,
        author_link=AUTHOR_LINK,
        notebook_link=NOTEBOOK_NAME,
        html_link=HTML_NAME,
    ))